In [1]:
!pip install langchain chromadb huggingface_hub transformers pandas langchain-community InstructorEmbedding google-generativeai python-dotenv
!pip install sentence_transformers==2.2.2

  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvji

In [2]:
!pip install huggingface_hub==0.25.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.28.1
    Uninstalling huggingface-hub-0.28.1:
      Successfully uninstalled huggingface-hub-0.28.1


In [11]:
import os
import pandas as pd
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
from InstructorEmbedding import INSTRUCTOR
import google.generativeai as genai
from dotenv import load_dotenv

In [10]:
import os
from google.colab import userdata

hf_token = userdata.get('hf_secret')
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hf_token

In [12]:
# --- Setup ---
load_dotenv()  # Load environment variables (e.g., your Gemini API key)


GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

# STT and TTS methods

In [47]:
!pip install SpeechRecognition
!pip install sounddevice
!pip install transformers soundfile TTS

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 37.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 34.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 50.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 938.0/938.0 kB 26.2 MB

In [20]:
import io
import json
import re
import requests
from transformers import pipeline
from IPython.display import Audio, display
from google.colab import files

In [22]:
# --- Setup STT using HuggingFace Whisper ---
stt_pipeline = pipeline("automatic-speech-recognition", model="openai/whisper-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


In [1]:
# # --- Setup TTS using ElevenLabs API ---
# ELEVENLABS_API_KEY = userdata.get('elevenlabs_key')
# ELEVENLABS_VOICE_ID = "21m00Tcm4TlvDq8ikWAM"  # Replace with your chosen voice ID
# ELEVENLABS_TTS_URL = f"https://api.elevenlabs.io/v1/text-to-speech/{ELEVENLABS_VOICE_ID}"
from TTS.api import TTS
# Using a pre-trained English model (you can change the model as needed)
tts_model = TTS(model_name="tts_models/en/ljspeech/tacotron2-DDC", progress_bar=False, gpu=False)

 > Downloading model to /root/.local/share/tts/tts_models--en--ljspeech--tacotron2-DDC
 > Model's license - apache 2.0
 > Check https://choosealicense.com/licenses/apache-2.0/ for more info.
 > Downloading model to /root/.local/share/tts/vocoder_models--en--ljspeech--hifigan_v2
 > Model's license - apache 2.0
 > Check https://choosealicense.com/licenses/apache-2.0/ for more info.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linea

/usr/local/lib/python3.11/dist-packages/TTS/utils/io.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_location, **kwargs)


 > Model's reduction rate `r` is set to: 1
 > Vocoder Model: hifigan
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Generator Model: hifigan_generator
 > Discriminator Model: hifigan_discriminator
Removing weight norm...


In [2]:
# def tts_to_audio(text):
#     """
#     Use ElevenLabs TTS API to synthesize speech from text and play the audio in Colab.
#     """
#     headers = {
#         "xi-api-key": ELEVENLABS_API_KEY,
#         "Content-Type": "application/json"
#     }
#     payload = {
#         "text": text,
#         "model_id": "eleven_turbo_v2",
#         "voice_settings": {"stability": 0.5, "similarity_boost": 0.8}
#     }
#     response = requests.post(ELEVENLABS_TTS_URL, headers=headers, json=payload)
#     if response.status_code == 200:
#         # Display the audio in Colab
#         display(Audio(response.content, autoplay=True))
#         print("TTS success")
#     else:
#         print("TTS error:", response.text)

def tts_to_audio(text):
    """Generate speech from text using Coqui TTS and play it in Colab."""
    output_file = "output.wav"
    tts_model.tts_to_file(text=text, file_path=output_file)
    display(Audio(output_file, autoplay=True))


In [25]:
chat_session()

Please upload an audio file containing your query (or type 'exit' in text input to end):


Saving Bob.opus to Bob (2).opus


/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


User (transcribed): Is Bob available for today?
Response Full: ```json
{
  "intent": "Yes",
  "query": "Check availability for Bob today"
}
```

History: 
User: Hi, how are you?
AI: Hello! How can I help you today?

A: I'm sorry, "today" is not a date I can use to check Dr. Bob's availability.  Please specify the date you'd like to book an appointment with Dr. Bob.
 > Text splitted to sentences.
['I\'m sorry, "today" is not a date I can use to check Dr. Bob\'s availability.', "Please specify the date you'd like to book an appointment with Dr. Bob."]
 > Processing time: 47.09056305885315
 > Real-time factor: 4.38431848503459


Please upload an audio file containing your query (or type 'exit' in text input to end):


...

Please upload an audio file containing your query (or type 'exit' in text input to end):


Saving yup.opus to yup.opus


/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


User (transcribed): Yeah, it's perfect.
Response Full: ```json
{
  "intent": "Yes",
  "query": "Find appointment with Dr. Bob on February 5th, earliest available appointment"
}
```

History: 
User: Hi, how are you?
AI: Hello! How can I help you today?
User: Is Bob available for today?
AI: I'm sorry, "today" is not a date I can use to check Dr. Bob's availability.  Please specify the date you'd like to book an appointment with Dr. Bob.
User: It's Bob, a level on 5th of February.
AI: Dr. Bob has appointments available on February 5th at 10:00 and 15:00.  What time would you prefer?
User: I would like to do it at 3pm.
AI: OK. So you'd like to book an appointment with Dr. Bob on February 5th at 3 PM.  Unfortunately, that time is not available.  Dr. Bob has appointments available at 10:00 AM and 1:00 PM on February 5th. Would you prefer either of those times?
User: Okay then, let's do it at any time.
AI: Dr. Bob has an appointment available on February 5th at 10:00 AM.  Would that work for 

Decoding json method's response: {"date": "2025-02-04", "doctor": "Dr. Bob", "time": "15:00"}
Decoded json: {'date': '2025-02-04', 'doctor': 'Dr. Bob', 'time': '15:00'}
Appointment updated in 'doctor_schedule_fixed.csv'
Results: {'ids': ['258fa0140005074e88a13165afa73aee716b34d6f7c0c676c7cb1715eeadb7b0'], 'embeddings': None, 'documents': ['SlotID: 258fa0140005074e88a13165afa73aee716b34d6f7c0c676c7cb1715eeadb7b0, Doctor: Dr. Bob, Date: 2025-02-04, Time: 15:00, Patient: Patient 84'], 'uris': None, 'data': None, 'metadatas': [{'Date': '2025-02-04', 'DoctorName': 'Dr. Bob', 'PatientName': 'Patient 84', 'SlotID': '258fa0140005074e88a13165afa73aee716b34d6f7c0c676c7cb1715eeadb7b0', 'Time': '15:00'}], 'included': [<IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}
SlotID: 258fa0140005074e88a13165afa73aee716b34d6f7c0c676c7cb1715eeadb7b0


In [4]:
def stt_from_file(audio_file_path):
    """
    Transcribe audio from the given file using the Whisper STT pipeline.
    """
    result = stt_pipeline(audio_file_path)
    return result.get("text", "").strip()

### Making of Collection && generate slot id

In [5]:
import pandas as pd
import hashlib
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceInstructEmbeddings

def generate_slot_id(doctor, date, time):
    # Create a deterministic hash based on key fields
    unique_str = f"{doctor}_{date}_{time}"
    return hashlib.sha256(unique_str.encode()).hexdigest()

def load_and_embed_data_unique(csv_file, collection_name="doctor_schedule_v2"):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)

    # Normalize time format (ensure consistency)
    df["Time"] = pd.to_datetime(df["Time"], format="%H:%M").dt.strftime("%H:%M")

    documents = []
    metadatas = []
    ids = []

    for _, row in df.iterrows():
        # Generate a unique SlotID using a hash of DoctorName, Date, and Time
        slot_id = generate_slot_id(row["DoctorName"], row["Date"], row["Time"])
        ids.append(slot_id)

        # Create structured text representation including the unique SlotID
        doc = (
            f"SlotID: {slot_id}, Doctor: {row['DoctorName']}, Date: {row['Date']}, "
            f"Time: {row['Time']}, Patient: {row['PatientName'] if pd.notna(row['PatientName']) else 'Available'}"
        )
        documents.append(doc)

        # Create metadata including the unique SlotID
        metadatas.append({
            "SlotID": slot_id,
            "DoctorName": row["DoctorName"],
            "Date": row["Date"],
            "Time": row["Time"],
            "PatientName": row["PatientName"] if pd.notna(row["PatientName"]) else "Available"
        })

    # Initialize the embedding model with improved instructions
    embedder = HuggingFaceInstructEmbeddings(
        model_name="BAAI/bge-large-en",
        embed_instruction="Represent the doctor's schedule entry uniquely, emphasizing DoctorName, Date, Time, and PatientName.",
        query_instruction="Retrieve the most relevant schedule by closely matching DoctorName, Date, and Time."
    )

    # Create the ChromaDB collection with documents, metadata, and unique IDs.
    # Using deterministic IDs ensures that reingesting duplicates won't add extra entries.
    vectordb = Chroma.from_texts(
        texts=documents,
        embedding=embedder,
        metadatas=metadatas,
        ids=ids,  # deterministic IDs prevent duplicates
        collection_name=collection_name
    )

    return vectordb

# Example usage:
csv_file_path = "doctor_schedule_fixed.csv"  # update if necessary
vectordb_unique = load_and_embed_data_unique(csv_file_path)
print("Data loaded with unique ingestion into vector store.")


/usr/local/lib/python3.11/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512
Data loaded with unique ingestion into vector store.


Query vector DB Method

In [6]:
def query_vector_database(vectordb, user_query, k=10):
    # Search for the k most similar documents to the user query
    results = vectordb.similarity_search(user_query, k=k)
    # Concatenate the retrieved document texts to form a context string
    context = "\n".join([doc.page_content for doc in results])
    return {"documents": context, "results": results}

# Example query (modify as needed)
user_query_example = "When is Dr. Bob available?"
retrieved_context = query_vector_database(vectordb_unique, user_query_example)
print("Retrieved Context:\n", retrieved_context["documents"])

Retrieved Context:
 SlotID: 4e97d4a10889fdb093bcab22f4b39376a6916ef6ad569fcbb6abcb2df0ce6b4c, Doctor: Dr. Bob, Date: 2025-02-01, Time: 09:00, Patient: Available
SlotID: d67443bd07be6dbca5cd3146da432e585b0cf84f9251624d099ed2d813bdbbe7, Doctor: Dr. Bob, Date: 2025-02-02, Time: 14:00, Patient: Available
SlotID: 8b70fbbd9bd43ccdbb842e0a545159c39691ae60a4f7fc953125ef958f727608, Doctor: Dr. Bob, Date: 2025-02-02, Time: 11:00, Patient: Available
SlotID: bdd418daedcf499d4e6296860b7bde4188655eb9ed3e6381c42ed670e0e55726, Doctor: Dr. Bob, Date: 2025-02-02, Time: 15:00, Patient: Available
SlotID: 2b076fe8c5a9612c319ade97e4e487cd36b8d9acb0b492aeee5323471d2a9681, Doctor: Dr. Bob, Date: 2025-02-02, Time: 10:00, Patient: Available
SlotID: 982e2fdb29ad474201503aae0d171721cb4b20cd8a87cf2a38eef3681e187c48, Doctor: Dr. Bob, Date: 2025-02-04, Time: 11:00, Patient: Patient 71
SlotID: c28640d175eedd6fda08a8d31e10d5427bc9e932d076654f68da27fb775c2a9e, Doctor: Dr. Bob, Date: 2025-02-05, Time: 10:00, Patient: Pa

Retrival intent and Details

In [7]:
def get_intent_and_query(chat_history):
    model = genai.GenerativeModel('gemini-1.5-flash')

    # Define prompt template
    prompt = f"""
    You are an AI assistant helping with doctor appointment bookings.
    Analyze the conversation history below and determine:

    1. Whether a database query is needed to fetch appointment details (respond "Yes" or "No").
    2. If "Yes", generate a **meaningful search query string** containing relevant details for searching the vector database.

    ## Chat History:
    {chat_history}

    ## Instructions:
    - If the user is **asking about appointment availability, booking, rescheduling, or confirming** an appointment, return intent "Yes".
    - If the user is just **greeting, making small talk, or asking general non-booking questions**, return intent "No".
    - If intent is "Yes":
        - **Extract the core intent** (e.g., checking availability, booking, modifying, or canceling).
        - **Include key details** (Doctor Name, Date, Time) if mentioned.
        - If no specific date is provided but the user asks about availability, assume the query is about "next available appointment."
        - If the user asks for **earliest availability**, include "earliest available appointment."
        - If the user is checking a **specific date and time**, ensure it's included in the query.
        - If rescheduling or modifying, include "reschedule" in the query.
        - If canceling an appointment, include "cancel appointment."
        - The final query should be **a well-formed, natural search string** that captures the exact request.

    ## Response Format (JSON):
    {{
        "intent": "Yes" or "No",
        "query": "A single search string summarizing the user request."
    }}
    """

    response = model.generate_content(prompt)

    response_text = response.candidates[0].content.parts[0].text

    print(f"Response Full: {response_text}")  # Debugging print

    # **Remove Markdown formatting (` ```json ... ``` `)**
    cleaned_response = re.sub(r"```json\n(.*?)\n```", r"\1", response_text, flags=re.DOTALL)

    try:
        # Convert to dictionary
        response_json = json.loads(cleaned_response)
        return response_json
    except json.JSONDecodeError as e:
        print(f"Error parsing LLM response: {e}")
        return {"intent": "No", "query": ""}

Chat History && LLM Prompt(Main)

In [8]:
# Initialize the Hugging Face generation pipeline
# generator = pipeline(
#     "text2text-generation",
#     model="google/flan-t5-xxl",
#     max_length=256
# )
# llm = HuggingFacePipeline(pipeline=generator)

# In-memory storage for user chat history (Replace with DB/Redis for persistence)
user_chat_history = {}

def get_user_chat_history(user_id):
    return user_chat_history.get(user_id, "")

def update_user_chat_history(user_id, user_input, ai_response):
    user_chat_history[user_id] = f"{user_chat_history.get(user_id, '')}\nUser: {user_input}\nAI: {ai_response}"

def generate_answer(user_id, user_query, retrieved_context):
    model = genai.GenerativeModel('gemini-1.5-flash')

    chat_history = get_user_chat_history(user_id)

    # Detect if it's a new conversation
    is_first_interaction = not bool(chat_history.strip())
    greeting = "Hello! How can I assist you with booking a doctor appointment today?\n\n" if is_first_interaction else ""

    # Construct a better prompt
    prompt = f"""
    You are an AI call agent assisting patients in booking doctor appointments. Your job is to provide structured, clear, and complete responses.
    Prioritize the information in the `Retrieved Schedule` when answering the query. The `Retrieved Schedule` contains the ONLY valid information for booking appointments.

    ## Conversation Context
    {chat_history}

    ## Retrieved Schedule
    {retrieved_context}

    ## Instructions:
    1. Only greet the user in the first message. If chat history exists, do NOT greet again.
    2. If the user has already provided a doctor name, date, or time in previous messages, do not ask again.
    3. If the user is trying to book an appointment but hasn't provided a time yet, ask for the time.
    4. If all required details are available, confirm the appointment in a structured way.

    ## User Query:
    {user_query}
    """

    # Generate response using Gemini
    response = model.generate_content(prompt)
    response_text = response.text.strip()
    print(f"History: {chat_history}")
    # Only prepend greeting if it's the first interaction AND the response doesn't already contain a greeting
    if is_first_interaction and not response_text.lower().startswith(("hello", "hi", "greetings")):
      response_text = greeting + response_text
    # Store conversation history
    update_user_chat_history(user_id, user_query, response_text)

    return response_text


Single Request Test

In [13]:
# 1. Ensure CSV is uploaded and available as 'doctor_schedule.csv'
# vectordb = load_and_embed_data("doctor_schedule.csv")

# 2. Query the vector store
user_query = "Dr. Bob 9:00 2025-02-05?"
retrieved_context = query_vector_database(vectordb_unique, user_query)
print("Retrieved Context:\n", retrieved_context["documents"])

# 3. Generate the final answer
final_answer = generate_answer(221,user_query, retrieved_context["documents"])
print(f"Q: {user_query}\nA: {final_answer}")


Retrieved Context:
 SlotID: c28640d175eedd6fda08a8d31e10d5427bc9e932d076654f68da27fb775c2a9e, Doctor: Dr. Bob, Date: 2025-02-05, Time: 10:00, Patient: Patient 42
SlotID: 4e97d4a10889fdb093bcab22f4b39376a6916ef6ad569fcbb6abcb2df0ce6b4c, Doctor: Dr. Bob, Date: 2025-02-01, Time: 09:00, Patient: Available
SlotID: 258fa0140005074e88a13165afa73aee716b34d6f7c0c676c7cb1715eeadb7b0, Doctor: Dr. Bob, Date: 2025-02-04, Time: 15:00, Patient: Patient 84
SlotID: b6fcd54f5540d751de02b076e65c38f2eb3ff48e6b1cbd2bc1b6fb52c3090178, Doctor: Dr. Bob, Date: 2025-02-05, Time: 15:00, Patient: Patient 41
SlotID: 982e2fdb29ad474201503aae0d171721cb4b20cd8a87cf2a38eef3681e187c48, Doctor: Dr. Bob, Date: 2025-02-04, Time: 11:00, Patient: Patient 71
SlotID: 36327ed32c2fbfe40b9e3692d6dec8654a4387ddc2f1bf50ca41311ceb9aa3d2, Doctor: Dr. Bob, Date: 2025-02-06, Time: 09:00, Patient: Patient 3
SlotID: 10397a58bd3f306391139c374d1770a30e290cb89325c8537f23c6698375c0b9, Doctor: Dr. Bob, Date: 2025-02-01, Time: 16:00, Patient:

Extract Appointment Info

In [14]:
#extract key info from llm
import json
import re
import typing_extensions as typing


class AppointmentInfo(typing.TypedDict):
    doctor: str
    date: str
    time: str


def extract_appointment_info(llm_response):
    model = genai.GenerativeModel(
        'gemini-1.5-flash',  # Or your preferred Gemini model
        generation_config=genai.GenerationConfig(
            temperature=0.1,  # Adjust as needed
            response_mime_type="application/json",
            response_schema=AppointmentInfo
        ))

    prompt = f"""
    Extract the doctor's name, date, and time from the following text and return them as a JSON object, convert time to 24-Hour format, date format to yyyy-mm-dd (if year not provided default as 2025):

    ```
    {llm_response}
    ```
    """

    response = model.generate_content(prompt)
    print(f"Decoding json method's response: {response.text}")  # Keep this for debugging

    try:
        appointment_info = json.loads(response.text)
        # appointment_info = response.text  # Directly access the JSON data
        print(f"Decoded json: {appointment_info}")
        return appointment_info
    except Exception as e:  # Catch any potential errors
        print(f"Error extracting appointment info: {e}")
        return None

Updating CSV with new Info

In [15]:
def update_appointment_csv(csv_file, appointment_info, patient_name):
    try:
        df = pd.read_csv(csv_file)
    except FileNotFoundError:
        print(f"Error: CSV file '{csv_file}' not found.")
        return

    if appointment_info:
        mask = (df['DoctorName'] == appointment_info['doctor']) & (df['Date'] == appointment_info['date']) & (df['Time'] == appointment_info['time'])
        if mask.any():
            df.loc[mask, 'PatientName'] = patient_name # Update the CSV
            try:
                df.to_csv(csv_file, index=False)
                print(f"Appointment updated in '{csv_file}'")
            except Exception as e:
                print(f"Error writing to CSV: {e}")
        else:
            print("No matching appointment found in the CSV.")
    else:
        print("No appointment information to update.")


Chat Session

In [24]:
user_id = 66
csv_file = "doctor_schedule_fixed.csv"
def chat_session():
    global chat_history_str # Declare chat_history_str as global
    chat_history_str = "" # Initialize it as an empty string

    while True:
        # Prompt user to upload an audio file (simulating voice input in Colab)
        print("Please upload an audio file containing your query (or type 'exit' in text input to end):")
        uploaded = files.upload()  # This will prompt a file upload dialog in Colab

        if not uploaded:
            user_input = input("No file uploaded. Type 'exit' to end or press Enter to try again: ")
            if user_input.lower() == "exit":
                break
            else:
                continue

        # Get filename of uploaded audio
        audio_filename = list(uploaded.keys())[0]

        # Convert the uploaded audio to text using STT
        user_query = stt_from_file(audio_filename)
        print(f"User (transcribed): {user_query}")

        if user_query.lower() in ["exit", "quit"]:
            break


        # user_query = input("Enter your query (or type 'exit' to end): ")
        # if user_query.lower() == "exit":
        #     break

        # Fetch chat history
        chat_history = get_user_chat_history(user_id)

        # **Append the latest user query** before sending it to LLM
        updated_chat_history = f"{chat_history}\nUser: {user_query}"

        # Get intent and details
        intent_details = get_intent_and_query(updated_chat_history)
        intent = intent_details.get("intent", "No")
        details = intent_details.get("details", {})

        # Retrieve context only when needed
        retrieved_context = query_vector_database(vectordb_unique, user_query) if intent == "Yes" else {"documents": []}

        final_answer = generate_answer(user_id, user_query, retrieved_context["documents"])
        print(f"\nA: {final_answer}")

        # Convert the final answer to speech using ElevenLabs TTS
        tts_to_audio(final_answer)

        # Accumulate chat history as a string.
        chat_history_str += f"User: {user_query}\nAI: {final_answer}\n"

        # Check for confirmation (example logic - adapt as needed)
        if "confirmed" in final_answer.lower() or "yes" in final_answer.lower() or "correct" in final_answer.lower() or "okay" in final_answer.lower(): # Basic confirmation check
            appointment_info = extract_appointment_info(chat_history_str)
            if appointment_info:
                update_appointment_csv(csv_file, appointment_info, user_id) #Pass user_id as patient name
                update_slot_metadata_by_details(vectordb_unique, appointment_info, user_id)

                break # Exit the loop after processing the confirmed booking
            else:
                print("Could not extract appointment information. Please provide the details again.")
                # You may want to reset chat_history_str here if extraction fails to re-prompt the user.
                chat_history_str = "" # Reset the chat history on extraction failure to restart the booking flow.
        elif "cancel" in final_answer.lower():
            print("Booking cancelled")
            chat_history_str = "" # Reset the chat history on cancel to restart the booking flow.
            break # Exit the loop after processing the confirmed booking

    # If the loop finished without confirmation it means the user exited without booking or cancelled
    else:
        print("Booking process incomplete")

Retrive Slot id

In [26]:
def get_slot_id(vectordb, doctor_name, date, time):
    # Combine multiple conditions using the "$and" operator
    results = vectordb.get(where={"$and": [
        {"DoctorName": doctor_name},
        {"Date": date},
        {"Time": time}
    ]})

    if results["ids"]:
        print(f"Results: {results}")
        # Iterate over the returned metadata entries to find one with a valid SlotID
        for meta in results["metadatas"]:
            slot_id = meta.get("SlotID")
            if slot_id:
                print(f"SlotID: {slot_id}")
                return slot_id
    return None
print(get_slot_id(vectordb_unique, "Dr. Bob", "2025-02-04", "15:00"))

Results: {'ids': ['258fa0140005074e88a13165afa73aee716b34d6f7c0c676c7cb1715eeadb7b0'], 'embeddings': None, 'documents': ['SlotID: 258fa0140005074e88a13165afa73aee716b34d6f7c0c676c7cb1715eeadb7b0, Doctor: Dr. Bob, Date: 2025-02-04, Time: 15:00, Patient: 66'], 'uris': None, 'data': None, 'metadatas': [{'Date': '2025-02-04', 'DoctorName': 'Dr. Bob', 'PatientName': 66, 'SlotID': '258fa0140005074e88a13165afa73aee716b34d6f7c0c676c7cb1715eeadb7b0', 'Time': '15:00'}], 'included': [<IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}
SlotID: 258fa0140005074e88a13165afa73aee716b34d6f7c0c676c7cb1715eeadb7b0
258fa0140005074e88a13165afa73aee716b34d6f7c0c676c7cb1715eeadb7b0


Updating Vector DB

In [18]:
#Updating vector db
from langchain.schema import Document

def update_slot_metadata_by_details(vectordb, appointment_info, new_patient_name):
    # Retrieve the SlotID using the helper function
    doctor_name = appointment_info['doctor']
    date = appointment_info['date']
    time = appointment_info['time']
    slot_id = get_slot_id(vectordb, doctor_name, date, time)
    if not slot_id:
        return "No matching slot found for the given details."

    # Now query using the SlotID
    results = vectordb.get(where={"SlotID": slot_id})
    if results["ids"]:
        doc_id = results["ids"][0]
        current_metadata = results["metadatas"][0]

        # Update metadata
        current_metadata["PatientName"] = new_patient_name

        # Create an updated document text
        doc_text = (
            f"SlotID: {slot_id}, Doctor: {doctor_name}, Date: {date}, "
            f"Time: {time}, Patient: {new_patient_name}"
        )

        # Create an updated Document object
        updated_doc = Document(page_content=doc_text, metadata=current_metadata)

        # Update the vector database
        vectordb.update_document(doc_id, updated_doc)

        return f"Slot {slot_id} updated: now booked for {new_patient_name}."

    return f"Slot {slot_id} not found in the vector DB."

# Example usage:
appointment = {
    'doctor': 'Dr. Carol',
    'date': '2025-02-05',
    'time': '09:00'
}
print(update_slot_metadata_by_details(vectordb_unique, appointment, "Patient 66"))
print(get_slot_id(vectordb_unique, "Dr. Carol", "2025-02-05", "09:00"))

Results: {'ids': ['5082bd7f8cca8b7c8720dde17132c24539e275b29f7cfd53d2467ea8691178eb'], 'embeddings': None, 'documents': ['SlotID: 5082bd7f8cca8b7c8720dde17132c24539e275b29f7cfd53d2467ea8691178eb, Doctor: Dr. Carol, Date: 2025-02-05, Time: 09:00, Patient: Available'], 'uris': None, 'data': None, 'metadatas': [{'Date': '2025-02-05', 'DoctorName': 'Dr. Carol', 'PatientName': 'Available', 'SlotID': '5082bd7f8cca8b7c8720dde17132c24539e275b29f7cfd53d2467ea8691178eb', 'Time': '09:00'}], 'included': [<IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}
SlotID: 5082bd7f8cca8b7c8720dde17132c24539e275b29f7cfd53d2467ea8691178eb
Slot 5082bd7f8cca8b7c8720dde17132c24539e275b29f7cfd53d2467ea8691178eb updated: now booked for Patient 66.
Results: {'ids': ['5082bd7f8cca8b7c8720dde17132c24539e275b29f7cfd53d2467ea8691178eb'], 'embeddings': None, 'documents': ['SlotID: 5082bd7f8cca8b7c8720dde17132c24539e275b29f7cfd53d2467ea8691178eb, Doctor: Dr. Carol, Date: 2025-02-05, Time: 09:00, 

Chat Session Method call

# LAngChAin OpenAi

In [ ]:
!pip install langchain_openai langchain-experimental

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain_openai import ChatOpenAI

In [ ]:
openai_key = userdata.get('openai_api')
os.environ["OPENAI_API_KEY"] = openai_key

In [ ]:
llm = ChatOpenAI(temperature=0.5)

In [ ]:
agent_executor = create_csv_agent(
    llm=llm,
    path="doctor_schedule_fixed.csv",
    verbose=True,
     allow_dangerous_code=True
)

In [ ]:
agent_executor.invoke("When is the earliest Dr. Alice avilable?")

# CHroma DB Exploration

In [ ]:
import chromadb
import numpy as np

client = chromadb.Client()
collection = client.get_collection("doctor_schedule")

# Get the dimensionality from an existing vector in your collection (if possible)
# Note: If your collection is empty, you'll have to know the dimensionality beforehand
if collection.count() > 0:
    sample_vector = collection.peek(limit=1)['embeddings'][0]
    vector_dimension = len(sample_vector)
else:
    # If the collection is empty you must provide the vector dimension
    vector_dimension = 1536 # Or whatever the correct dimension is

# Generate a random query vector with the correct dimensionality
query_vector = np.random.rand(vector_dimension).astype(np.float32)

results = collection.query(query_embeddings=[query_vector], n_results=10, include=["documents", "metadatas"])

# Now you can inspect the results
for i in range(len(results['documents'])):
    print(f"Result {i+1}:")
    print(f"Document: {results['documents'][i]}")
    print(f"Metadata: {results['metadatas'][i]}")
    print("-" * 20)